## Важно:

Пожалуйста, поддерживайте ваш код в хорошем состоянии, пишите комментарии, убирайте бесполезные ячейки, пишите модели в специально отведенных модулях. Проверяющие могут **НА СВОЕ УСМОТРЕНИЕ** снижать баллы за:

1. Говнокод
2. Неэффективные решения
3. Вермишель из ячеек в тетрадке
4. Все остальное что им не понравилось


In [ ]:
#!XL
import wandb

## Важно 2 (0 - 0.15 балла):

За использование логгеров типа wandb/comet/neptune и красивую сборку этой домашки в виде графиков/картинок в этих логгерах мы будем выдавать бонусные баллы.



## Важно 3:

Решением домашки является архив с использованными тетрадками/модулями, а так же **.pdf файл** с отчетом по проделанной работе по каждому пункту задачи. 
В нем необходимо описать какие эксперименты вы производили чтобы получить результат который вы получили, а так же обосновать почему вы решили использовать штуки которые вы использовали (например дополнительные лоссы для стабилизации, WGAN-GP, а не GAN/WGAN+clip)


In [2]:
#!XL
import numpy as np
import torchvision

import torch
from torch import nn
from torch.nn import functional as F

from tqdm.notebook import tqdm
from IPython.display import clear_output


import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#!XL
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    


download_file_from_google_drive('1F96x4LDbsTZGMMq81fZr7aduJCe8N95O', 'celeba.zip')

KeyboardInterrupt: 

In [3]:
#!XL
%enable_full_walk

In [ ]:
#!XL
import zipfile
with zipfile.ZipFile('celeba.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

В этом домашнем задании мы будем работать с Celeba. Celeba - это уже известный вам датасет состоящий из фотографий селеб в их привычной местности:

In [34]:
#!XL
image_size=(128, 128)
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(image_size),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [35]:
#!XL
celeba = torchvision.datasets.CelebA('celeba', target_type='attr', transform=transforms, download=False)
celeba_dataloader = torch.utils.data.DataLoader(celeba, 64, shuffle=True, num_workers=32)

In [6]:
#!XL
index2attr = {i:j for i, j in enumerate(celeba.attr_names)}

In [7]:
#!XL
from importlib import reload

In [30]:
#!XL
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
import model as model_

In [31]:
#!XL
reload(model_)

<module 'model' from '/home/jupyter/work/resources/DeepGenerativeModels/homework/2-GAN/model.py'>

In [16]:
#!XL
x, y = next(iter(celeba_dataloader))

In [ ]:
#!XL
trainer = Trainer(logger=WandbLogger(project='GAN-homework_2-GAN', save_dir=None), log_every_n_steps=1, gpus=-1, accelerator='dp')
model__ = model_.VanillaStarGAN(x, y, index2attr)

In [ ]:
#!XL
__spec__ = None
trainer.fit(model__, celeba_dataloader, celeba_dataloader)

In [14]:
#!XL
%%debug

ipdb>  locals()


NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

{'nn': <module 'torch.nn' from '/usr/local/lib/python3.7/dist-packages/torch/nn/__init__.py'>, '_oh': {9: <module 'model' from '/home/jupyter/work/resources/DeepGenerativeModels/homework/2-GAN/model.py'>}, '_i8': '\nfrom pytorch_lightning import Trainer\nfrom pytorch_lightning.loggers import WandbLogger\nimport model as model_\n#', '_9': <module 'model' from '/home/jupyter/work/resources/DeepGenerativeModels/homework/2-GAN/model.py'>, '___': '', '_i1': '\nimport wandb\n#', '_ih': ['', 'import wandb\n#', "import numpy as np\nimport torchvision\n\nimport torch\nfrom torch import nn\nfrom torch.nn import functional as F\n\nfrom tqdm.notebook import tqdm\nfrom IPython.display import clear_output\n\n\nimport matplotlib.pyplot as plt\nget_ipython().run_line_magic('matplotlib', 'inline')\n#", "get_ipython().run_line_magic('enable_full_walk', '')\n#", 'image_size=(128, 128)\ntransforms = torchvision.transforms

ipdb>  exit


В этой домашней работе вам предлагается повторить результаты статьи StarGAN (https://arxiv.org/abs/1711.09020). 

Основная часть домашнего задания - чтение статьи и улучшение результатов, поэтому обязательно прочитайте не только StarGAN, но и другие Image-to-Image GAN подходы того времени (17-18 год) 


## Задача 1 (0.4 балла):

Повторить результаты StarGAN используя только CelebA

что это значит: в статье предлагается способ использовать несколько датасетов и выучивание аттрибутов уникальных для какого-то одного датасета. Мы не просим вас это делать, вам достаточно просто обучить StarGAN на CelebA

In [82]:
#!L
model

In [ ]:
from model import StarGAN
from utils import permute_labels

model = StarGAN()

In [ ]:
model.train()
for image, label in tqdm(celeba_dataloader, leave=False, desc=f"trainloop: {epoch}"):
    # YOUR CODE

model.eval()
for ind, (image, label) in enumerate(celeba_val_dataloader): # batch = 1
    if ind >= 10: break

    # пример сравнения качества на глаз:
    new_label = permute_labels(label)
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.imshow((image[0].permute(1, 2, 0) + 1) / 2)
    plt.subplot(1, 2, 2)
    fake_image = model.generate(image.to(device), new_label.to(device)).detach().cpu()[0]
    plt.imshow((fake_image.permute(1, 2, 0) + 1) / 2)
    plt.show()


## Важно 4: 

Если вы учите на колабе или на наших машинках, вероятнее всего что обучение будет очень долгим на картинках 256х256. Никто не мешает уменьшить разрешение, главное чтобы было видно что трансформации выучились

Еще, кажется что не все аттрибуты селебы являются очень важными или достаточно представленными в датасете. Не запрещается убирать бесполезные аттрибуты (только обоснуйте почему так сделали в отчете)

Не забывайте про аугментации

## Важно 5: 

Да, мы знаем что в на гитхабе лежить готовый код на путорче для этой статьи. Проблема в том что он написал на torch 0.4, поэтому, если мы увидим что вы используете __старый__ код со старыми модулями, то мы:

1. Будем неодобрительно смотреть
2. За наглое списывание будем снимать баллы


## Задача 2 (0.2 балла): 

Мерить качество на глаз - плохая идея. Подключите подсчет FID для каждой N эпохи, чтобы вы могли следить за прогрессом модели.

Сранение моделей между собой тоже возможно только по FID, поэтому трекайте его когда будете делать другие эксперименты

## Задача 3 (0.4 балла):

Если вы будете дословно повторять архитектуру авторов статьи, вы сразу же увидите что обучение станет дико долгим и не очень стабильным. Возможно у вас получится предложить несколько улучшений, которые приведут к хорошему FID, к визуально лучшим результатам или к более стабильному обучению.

В этой задаче хочется чтобы вы попробовали улучшить результаты статьи используя либо то что уже знаете, либо что-то из релевантных статей по Im2Im современности

## Важно 6: 

Когда вы будете показывать визуальные трансформации которые делает ваш StarGAN, хорошей идеей будет сразу же зафиксировать набор картинок (очевидно из валидации) и набор трансформаций на которых вы будете показывать результаты. Например: 10 картинок разных людей на которых вы покажете Male-Female, Beard-noBeard, Old-Young трансформации

## Важно 7 (0.15 балла): 

Выдам дополнительные баллы если у вас получится визуально красивая перекраска волос в разные цвета